# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
dataset_path = "Data_Repository/"

# Load dataset

In [3]:
df_items = pd.read_csv(dataset_path+'items.csv')
print("Shape: ", df_items.shape)
df_items.head(3)

Shape:  (250, 25)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,NaN,98000000,Action;Adventure,NaN,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,10017322,119,en;la,Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,5.7,137,exotic island;treasure;map;ship;scalp;pirate
1,False,NaN,4000000,Crime;Comedy,NaN,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,4300000,98,en,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539,hotel;new year's eve;witch;bet;hotel room;sper...
2,False,NaN,0,Music;Drama;Family,NaN,2054,tt0113862,en,Mr. Holland's Opus,"In 1965, passionate musician Glenn Holland tak...",...,106269971,137,en,Released,"Of All the Lives He Changed, the One That Chan...",Mr. Holland's Opus,False,6.9,115,composer;mentor;deaf-mute;musical;apprentice;p...


In [4]:
df_items.iloc[0]

adult                                                                False
belongs_to_collection                                                  NaN
budget                                                            98000000
genres                                                    Action;Adventure
homepage                                                               NaN
id                                                                    1408
imdb_id                                                          tt0112760
original_language                                                       en
original_title                                            Cutthroat Island
overview                 Morgan Adams and her slave, William Shaw, are ...
popularity                                                        7.284477
poster_path                               /odM9973kIv9hcjfHPp6g6BlyTIJ.jpg
production_companies     Le Studio Canal+;Laurence Mark Productions;Met...
production_countries     

In [5]:
df_ratings = pd.read_csv(dataset_path+'ratings.db.csv')
print("Shape: ", df_ratings.shape)
df_ratings.head(3)

Shape:  (51956, 3)


,userId,itemId,rating
0,50,296,3.0
1,50,318,4.0
2,50,480,3.0


In [6]:
df_ratings_known = pd.read_csv(dataset_path+'ratings.test.known.csv')
print("Shape: ", df_ratings_known.shape)
df_ratings_known.head(3)

Shape:  (3018, 3)


,userId,itemId,rating
0,125328,8961,4.0
1,79268,4226,4.0
2,79268,1732,5.0


In [7]:
df_ratings_unknown = pd.read_csv(dataset_path+'ratings.test.unknown.csv')
print("Shape: ", df_ratings_unknown.shape)
df_ratings_unknown.head(3)

Shape:  (1000, 3)


,userId,itemId,rating
0,125328,58559,4.5
1,125328,1246,4.5
2,125328,3578,4.0


In [8]:
df_ratings_all = pd.concat([df_ratings, df_ratings_known])
print("Shape: ", df_ratings_all.shape)
df_ratings_all.head(3)

Shape:  (54974, 3)


,userId,itemId,rating
0,50,296,3.0
1,50,318,4.0
2,50,480,3.0


# ONE SLOPE

In [9]:
# rating_matrix = df_ratings_all.pivot_table(index='userId', columns='itemId', values='rating')
# rating_matrix

In [10]:
# def get_dev_val(dataframe, i, j):
#     dataframe = dataframe[
#         (dataframe[i].notnull())
#         & (dataframe[j].notnull())
#     ].copy()
#     users = len(dataframe)
#     if users > 0: dev_val = (rating_matrix[i]-rating_matrix[j]).sum() / users
#     else: dev_val = 0
#     return dev_val

# deviations = {}
# for i in rating_matrix.columns:
#     for j in rating_matrix.columns:
#         if i == j:
#             continue
#         deviation = get_dev_val(rating_matrix, i, j)
#         deviations[i, j] = deviation
#         deviations[j, i] = (-1) * deviation

# deviations

In [11]:
# index = list(set([k[0] for k in deviations.keys()]))
# index.sort()
# columns = list(set([k[1] for k in deviations.keys()]))
# columns.sort()

# dataframe_desviation = pd.DataFrame(index=index, columns=columns)

# for k, v in deviations.items():
#     dataframe_desviation.loc[k[0], k[1]] = v

In [12]:
# dataframe_desviation

In [13]:
# user_to_predict = df_ratings_known['userId'].unique().tolist()

# predictions = {}
# for u in user_to_predict:
#     user_row = rating_matrix.loc[u, :]
#     user_ranked_items = user_row[user_row.notnull()].index
#     for i in dataframe_desviation.columns:
#         predictions[u, i] = (np.sum(dataframe_desviation.loc[i, user_ranked_items] + rating_matrix.loc[u, user_ranked_items])) / len(user_ranked_items)

# predictions

In [14]:
# index = list(set([k[0] for k in predictions.keys()]))
# index.sort()
# columns = list(set([k[1] for k in predictions.keys()]))
# columns.sort()

# dataframe_prediction = pd.DataFrame(index=index, columns=columns)

# for k, v in predictions.items():
#     dataframe_prediction.loc[k[0], k[1]] = v

# dataframe_prediction

In [15]:
# best_predictions = {}

# for idx, row in dataframe_prediction.iterrows():
#     top_10 = row.sort_values(ascending=False).head(10)
#     best_predictions[idx] = list(zip(top_10.index, top_10.values))

# best_predictions

# CLUSTERING

In [16]:
df_clustering =  df_ratings_all.groupby(by=['userId']).agg({'itemId': 'count', 'rating' : 'mean'}).reset_index()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_clustering[["itemId","rating"]])
df_clustering[["itemId","rating"]] = pd.DataFrame(data=scaled_data, columns=["itemId","rating"])
df_clustering

,userId,itemId,rating
0,50,-0.529856,-0.308372
1,151,0.927547,-0.890308
2,162,-0.063487,-0.275855
3,260,-0.588153,1.675144
4,420,-0.617301,1.310957
...,...,...,...
2095,270477,2.180915,-0.629997
2096,270485,-0.121783,0.942041
2097,270590,-0.529856,0.959777
2098,270801,-0.413264,-0.731089


In [17]:
kmeans = KMeans(n_clusters=4, random_state=1234)
kmeans.fit(df_clustering[["itemId","rating"]])
cluster_labels = kmeans.labels_

c:\Users\RodrigoRenatoPomaLud\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [18]:
df_user_clustered = df_clustering[["userId"]].drop_duplicates()
df_user_clustered["cluster"] = cluster_labels
df_user_clustered

,userId,cluster
0,50,0
1,151,0
2,162,0
3,260,1
4,420,1
...,...,...
2095,270477,3
2096,270485,1
2097,270590,1
2098,270801,0


In [19]:
for cluster in df_user_clustered['cluster'].unique():
    quantity = df_user_clustered[df_user_clustered['cluster'] == cluster]['userId'].count()
    print("The cluster {} has {} users".format(cluster, quantity))

The cluster 0 has 911 users
The cluster 1 has 724 users
The cluster 3 has 188 users
The cluster 2 has 277 users


# ONE SLOPE CLUSTERED

In [20]:
def get_matrix(dictionary):
    index = list(set([k[0] for k in dictionary.keys()]))
    index.sort()
    columns = list(set([k[1] for k in dictionary.keys()]))
    columns.sort()

    dataframe = pd.DataFrame(index=index, columns=columns)

    for k, v in dictionary.items():
        dataframe.loc[k[0], k[1]] = v
    return dataframe

def one_slope(dataframe):
    rating_matrix = dataframe.pivot_table(index='userId', columns='itemId', values='rating')
    def get_dev_val(dataframe, i, j):
        dataframe = dataframe[
            (dataframe[i].notnull())
            & (dataframe[j].notnull())
        ].copy()
        users = len(dataframe)
        if users > 0: dev_val = (rating_matrix[i]-rating_matrix[j]).sum() / users
        else: dev_val = 0
        return dev_val

    deviations = {}
    for i in rating_matrix.columns:
        for j in rating_matrix.columns:
            if i == j:
                continue
            deviation = get_dev_val(rating_matrix, i, j)
            deviations[i, j] = deviation
            deviations[j, i] = (-1) * deviation

    desviation_matrix = get_matrix(deviations)
    return rating_matrix, desviation_matrix

In [21]:
matrix_per_cluster = {}
for cluster in df_user_clustered['cluster'].unique():
    matrix = {}
    users_clustered = df_user_clustered[df_user_clustered['cluster'] == cluster]['userId'].tolist()
    df_ratings_clustered = df_ratings_all[df_ratings_all['userId'].isin(users_clustered)]
    matrix['rating_matrix'], matrix['desviation_matrix'] = one_slope(df_ratings_clustered)
    matrix_per_cluster[cluster] = matrix

In [22]:
matrix_per_cluster

{0: {'rating_matrix': itemId  2      3      5      6      11     16     17     19     21     25     \
  userId                                                                         
  50        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
  151       NaN    3.0    NaN    NaN    3.0    NaN    NaN    2.0    4.0    NaN   
  162       4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    5.0    NaN   
  421       5.0    NaN    NaN    NaN    4.0    NaN    NaN    NaN    NaN    NaN   
  1307      NaN    NaN    NaN    NaN    NaN    NaN    1.0    NaN    NaN    NaN   
  ...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
  269484    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
  270136    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    NaN    NaN   
  270257    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
  270801    NaN    NaN    NaN    NaN    NaN    4.5    NaN    NaN    NaN    NaN

In [23]:
user_to_predict = df_ratings_known['userId'].unique().tolist()

predictions = {}
for u in user_to_predict:
    user_cluster = df_user_clustered[df_user_clustered['userId'] == u]['cluster'].values[0]
    user_matrix = matrix_per_cluster[user_cluster]
    user_rating_matrix = user_matrix['rating_matrix']
    user_desviation_matrix = user_matrix['desviation_matrix']
    user_row = user_rating_matrix.loc[u, :]
    user_ranked_items = user_row[user_row.notnull()].index
    for i in user_desviation_matrix.columns:
        if i not in user_ranked_items:
            predictions[u, i] = (np.sum(user_rating_matrix.loc[u, user_ranked_items] + user_desviation_matrix.loc[i, user_ranked_items])) / len(user_ranked_items)
predictions_matrix = get_matrix(predictions)

In [24]:
predictions_matrix

,2,3,5,6,11,16,17,19,21,25,...,8961,30707,40815,44191,45722,46578,48780,58559,68954,74458
2378,1.166667,1.875,1.875,2.333333,2.875,2.5,2.4375,1.791667,1.333333,1.625,...,2.5,2.375,3.125,2.5,3.375,2.25,3.5625,2.5,2.25,2.75
7238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.81447,NaN,NaN,...,4.199741,4.214281,3.697227,4.131496,3.583648,4.103869,4.356277,4.553387,4.026719,4.041915
10218,3.48786,3.182481,3.627837,3.950496,3.855098,4.147852,4.268815,NaN,3.600107,3.723454,...,NaN,3.848325,NaN,3.679495,NaN,3.896599,4.017983,4.135129,4.162733,3.88772
11418,1.40625,NaN,2.302083,2.59375,2.75,3.0625,3.803571,0.0625,2.25,1.732143,...,2.5625,4.875,2.75,3.0,2.6875,2.375,2.6875,2.875,2.875,3.25
15238,NaN,2.959829,2.565934,2.843534,3.174908,3.104701,3.25989,NaN,3.117308,2.48014,...,3.010897,3.040598,3.461538,3.660897,3.367521,2.532051,3.313034,3.024573,3.605128,3.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243538,3.583925,3.05787,3.750418,3.841859,3.88328,3.834887,4.639543,2.660254,3.835684,3.810084,...,3.767806,3.830314,3.874398,3.770455,3.683868,3.811036,4.322569,4.141186,3.900253,3.935307
247588,3.265339,2.789503,3.322393,3.610883,3.539792,3.604529,4.374514,2.526158,NaN,3.611242,...,3.828136,3.634682,3.430293,3.261403,3.492847,3.733924,3.636925,3.699991,3.585339,3.724412
249158,3.437656,2.861634,3.131247,3.59987,3.621702,3.628337,3.908794,2.492781,3.610991,3.2881,...,3.834288,4.04367,4.156537,3.809963,3.674265,3.816491,3.988513,4.144573,4.076723,4.039165
259448,3.408279,3.106365,3.44701,3.804982,3.757569,3.747432,4.412884,2.672595,NaN,3.707007,...,4.027865,3.971637,3.808514,3.585088,3.966366,3.795163,3.886249,4.108912,4.033952,4.017494


In [25]:
best_predictions = {}

for idx, row in predictions_matrix.iterrows():
    top_10 = row.sort_values(ascending=False).head(10)
    best_predictions[idx] = list(zip(top_10.index, top_10.values))

best_predictions

{2378: [(4973, 4.375),
  (2959, 4.28125),
  (2502, 4.0),
  (4011, 3.6875),
  (4896, 3.625),
  (48780, 3.5625),
  (1090, 3.5),
  (3114, 3.5),
  (1584, 3.4375),
  (8368, 3.375)],
 7238: [(750, 4.622245262109247),
  (913, 4.572360609594698),
  (260, 4.565541668010004),
  (58559, 4.553386546200975),
  (293, 4.470434264776243),
  (903, 4.437444346769766),
  (6016, 4.436938340051617),
  (1252, 4.39139074083162),
  (541, 4.371177501267554),
  (745, 4.367322124148139)],
 10218: [(2019, 4.651292200269183),
  (1252, 4.583760875092583),
  (6016, 4.58103354041583),
  (903, 4.50342011085345),
  (318, 4.49786222494792),
  (913, 4.38775622555194),
  (1213, 4.366615133851825),
  (5995, 4.364689610644433),
  (750, 4.355110284419506),
  (296, 4.354648994975047)],
 11418: [(3578, 5.1875),
  (2023, 5.0),
  (30707, 4.875),
  (858, 4.8125),
  (1961, 4.533333333333333),
  (111, 4.5),
  (58, 4.5),
  (17, 3.803571428571429),
  (527, 3.7395833333333335),
  (1393, 3.7083333333333335)],
 15238: [(1732, 4.17414529

In [26]:
best_predictions_matrix_list = []
best_predictions_matrix = pd.DataFrame.from_dict(best_predictions, orient='index')
best_predictions_matrix = best_predictions_matrix.reset_index()
best_predictions_matrix.rename(columns={'index':'userId'}, inplace=True)

for i in range(10):
        best_predictions_matrix_list.append(best_predictions_matrix[['userId', i]].rename(columns={i : 'items'}))

best_predictions_matrix = pd.concat(best_predictions_matrix_list, ignore_index=True)
best_predictions_matrix[['itemId', 'rating']] = best_predictions_matrix['items'].apply(pd.Series)
best_predictions_matrix.drop(columns=['items'], inplace=True)


best_predictions_matrix = best_predictions_matrix.sort_values(by=['userId', 'rating'], ascending=False).reset_index(drop=True)
best_predictions_matrix['itemId'] = best_predictions_matrix['itemId'].astype(int).astype(str)
best_predictions_matrix['userId'] = best_predictions_matrix['userId'].astype(str)

best_predictions_matrix[['itemId', 'userId']].groupby(by=['itemId'], as_index=False).count().sort_values(by=['userId'], ascending=False)

,itemId,userId
48,2019,51
136,912,41
106,5618,40
12,1252,40
125,750,39
...,...,...
105,552,1
35,1639,1
72,3114,1
34,163,1


# Evaluación

In [27]:
def discounted_cumulative_gain(recommended_list, ground_truth_list):
    dcg_value = 0
    for i in range(len(recommended_list)):
        recommended_item = recommended_list.loc[i, 'itemId']
        if recommended_item in ground_truth_list['itemId'].tolist():
            item_position = ground_truth_list[ground_truth_list['itemId'] == recommended_list.loc[i, 'itemId']].index[0]
            item_rating = ground_truth_list.loc[item_position, 'rating']
            dcg_value += item_rating / np.log2(i + 2) #se agrega un uno adicional porque empieza en 0
    return dcg_value

def ideal_discounted_cumulative_gain(recommended_list, ground_truth_list):
    idcg_value = 0
    for i in range(len(recommended_list)):
        recommended_item = recommended_list.loc[i, 'itemId']
        if recommended_item in ground_truth_list['itemId'].tolist():
            item_position = ground_truth_list[ground_truth_list['itemId'] == recommended_list.loc[i, 'itemId']].index[0]
            item_rating = ground_truth_list.loc[item_position, 'rating']
            idcg_value += item_rating / np.log2(item_position + 2) #se agrega un uno adicional porque empieza en 0
    return idcg_value

In [42]:
def discounted_cumulative_gain_v2(recommended_list, ground_truth_list):
    dcg_value = 0
    asserts = recommended_list[recommended_list['itemId'].isin(ground_truth_list['itemId'])]['itemId'].tolist()
    for assert_item in asserts:
        gt_item_position = ground_truth_list[ground_truth_list['itemId'] == assert_item].index[0]
        r_item_position = recommended_list[recommended_list['itemId'] == assert_item].index[0]
        gt_item_rating = ground_truth_list.loc[gt_item_position, 'rating']
        dcg_value += gt_item_rating / np.log2(r_item_position + 2) #se agrega un uno adicional porque empieza en 0
    return dcg_value

def ideal_discounted_cumulative_gain_v2(ground_truth_list):
    idcg_value = 0
    for item in ground_truth_list.index:
        idcg_value += ground_truth_list.loc[item, 'rating'] / np.log2(item + 2) #se agrega un uno adicional porque empieza en 0
    return idcg_value

In [43]:
qualitive_measures = {}

for user in df_ratings_unknown['userId'].unique():
    df_user_ground_truth = df_ratings_unknown[df_ratings_unknown['userId'] == user][['itemId', 'rating']].sort_values(by=['rating', 'itemId'], ascending=False).reset_index(drop=True)
    df_user_recommended = pd.DataFrame(best_predictions[user], columns=['itemId', 'rating']).reset_index(drop=True)
    asserts = df_user_recommended['itemId'].isin(df_user_ground_truth['itemId']).sum()
    if asserts == 0: dcg, idcg, ndcg = 0, 0, 0
    else:
        dcg = discounted_cumulative_gain_v2(df_user_recommended, df_user_ground_truth)
        idcg = ideal_discounted_cumulative_gain_v2(df_user_ground_truth)
        ndcg = dcg / idcg if idcg != 0 else 0
    qualitive_measures[user] = (asserts, dcg, idcg, ndcg)
qualitive_measures

{125328: (0, 0, 0, 0),
 79268: (0, 0, 0, 0),
 152978: (0, 0, 0, 0),
 171648: (0, 0, 0, 0),
 227648: (2, 3.2618591223240356, 17.26409848077655, 0.18893886211064495),
 16498: (0, 0, 0, 0),
 166008: (1, 3.1546487678572874, 19.390704141080757, 0.16268871645428862),
 159588: (1, 2.0, 21.645570325007462, 0.09239765781035435),
 120238: (0, 0, 0, 0),
 93998: (2, 4.401373922735365, 18.9904640454219, 0.23176758146657403),
 130768: (0, 0, 0, 0),
 178358: (1, 1.4453241315894392, 21.372657167524252, 0.06762491534209461),
 121578: (1, 1.505149978319906, 18.777209247506022, 0.08015834294011605),
 243538: (0, 0, 0, 0),
 171548: (0, 0, 0, 0),
 120718: (1, 1.75, 19.60238725654104, 0.08927484071696674),
 2378: (0, 0, 0, 0),
 11418: (1, 1.934264036172708, 21.197584515042532, 0.0912492663869361),
 165718: (0, 0, 0, 0),
 78488: (0, 0, 0, 0),
 170998: (1, 0.9030899869919436, 20.888808836358926, 0.04323319697483329),
 225148: (1, 0.3010299956639812, 18.411022772157065, 0.016350530841731832),
 103768: (3, 4.75

In [44]:
df_qualitive_measures = pd.DataFrame(
                            data=qualitive_measures.values(), 
                            columns=['asserts', 'dcg', 'idcg', 'ndcg'],
                            index=qualitive_measures.keys()
                        )
df_qualitive_measures

,asserts,dcg,idcg,ndcg
125328,0,0.000000,0.000000,0.000000
79268,0,0.000000,0.000000,0.000000
152978,0,0.000000,0.000000,0.000000
171648,0,0.000000,0.000000,0.000000
227648,2,3.261859,17.264098,0.188939
...,...,...,...,...
166408,0,0.000000,0.000000,0.000000
125668,1,1.354635,17.436075,0.077692
21608,1,1.750000,13.343875,0.131146
7238,0,0.000000,0.000000,0.000000


In [45]:
df_qualitive_measure_asserts = df_qualitive_measures[df_qualitive_measures['asserts'] != 0]
print("Se agrupan los ususarios en {} CLUSTERS".format(len(matrix_per_cluster.keys())))
print("Usuarios con una recomendación minima de una pelicula acertada {} de {}".format(df_qualitive_measure_asserts['asserts'].count(),df_qualitive_measures.shape[0]))
print("Porcentaje de usuarios con una recomendación minima de una pelicula acertada {}%".format(100*df_qualitive_measure_asserts['asserts'].count()/df_qualitive_measures.shape[0]))
print("Peliculas recomendadas acertivamente {} de {}".format(df_qualitive_measure_asserts['asserts'].sum(),(df_qualitive_measures.shape[0]*10)))
print("Porcentaje de peliculas recomendadas acertivamente {}%".format(100*df_qualitive_measure_asserts['asserts'].sum()/(df_qualitive_measures.shape[0]*10)))
print("Promedio de Normalized Discount Cumulative Gain (nDCG) {}".format(df_qualitive_measures['ndcg'].sum()/(df_qualitive_measures.shape[0])))

Se agrupan los ususarios en 4 CLUSTERS
Usuarios con una recomendación minima de una pelicula acertada 47 de 100
Porcentaje de usuarios con una recomendación minima de una pelicula acertada 47.0%
Peliculas recomendadas acertivamente 67 de 1000
Porcentaje de peliculas recomendadas acertivamente 6.7%
Promedio de Normalized Discount Cumulative Gain (nDCG) 0.0685225977739509


In [32]:
df_test = df_qualitive_measures[['asserts']].copy()
df_test['userId'] = df_test.index.values

df_test_cluster = df_user_clustered[df_user_clustered['userId'].isin(df_ratings_unknown['userId'].unique().tolist())]

df_test_cluster_merged = pd.merge(
    left=df_test_cluster,
    right=df_test,
    on=['userId'],
    how='inner'
)
#df_test_cluster.groupby(['cluster'], as_index=False).count()
df_test_cluster_merged[['cluster', 'asserts']].groupby(['cluster'], as_index=False).sum()

,cluster,asserts
0,0,25
1,1,22
2,2,8
3,3,12
